# --- Day 16: The Floor Will Be Lava ---
With the beam of light completely focused somewhere, the reindeer leads you deeper still into the Lava Production Facility. At some point, you realize that the steel facility walls have been replaced with cave, and the doorways are just cave, and the floor is cave, and you're pretty sure this is actually just a giant cave.

Finally, as you approach what must be the heart of the mountain, you see a bright light in a cavern up ahead. There, you discover that the beam of light you so carefully focused is emerging from the cavern wall closest to the facility and pouring all of its energy into a contraption on the opposite side.

Upon closer inspection, the contraption appears to be a flat, two-dimensional square grid containing empty space (.), mirrors (/ and \), and splitters (| and -).

The contraption is aligned so that most of the beam bounces around the grid, but each tile on the grid converts some of the beam's light into heat to melt the rock in the cavern.

You note the layout of the contraption (your puzzle input). For example:
```
.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....
```
The beam enters in the top-left corner from the left and heading to the right. Then, its behavior depends on what it encounters as it moves:

If the beam encounters empty space (.), it continues in the same direction.
If the beam encounters a mirror (/ or \), the beam is reflected 90 degrees depending on the angle of the mirror. For instance, a rightward-moving beam that encounters a / mirror would continue upward in the mirror's column, while a rightward-moving beam that encounters a \ mirror would continue downward from the mirror's column.
If the beam encounters the pointy end of a splitter (| or -), the beam passes through the splitter as if the splitter were empty space. For instance, a rightward-moving beam that encounters a - splitter would continue in the same direction.
If the beam encounters the flat side of a splitter (| or -), the beam is split into two beams going in each of the two directions the splitter's pointy ends are pointing. For instance, a rightward-moving beam that encounters a | splitter would split into two beams: one that continues upward from the splitter's column and one that continues downward from the splitter's column.
Beams do not interact with other beams; a tile can have many beams passing through it at the same time. A tile is energized if that tile has at least one beam pass through it, reflect in it, or split in it.

In the above example, here is how the beam of light bounces around the contraption:
```
>|<<<\....
|v-.\^....
.v...|->>>
.v...v^.|.
.v...v^...
.v...v^..\
.v../2\\..
<->-/vv|..
.|<<<2-|.\
.v//.|.v..
```
Beams are only shown on empty tiles; arrows indicate the direction of the beams. If a tile contains beams moving in multiple directions, the number of distinct directions is shown instead. Here is the same diagram but instead only showing whether a tile is energized (#) or not (.):
```
######....
.#...#....
.#...#####
.#...##...
.#...##...
.#...##...
.#..####..
########..
.#######..
.#...#.#..
```
Ultimately, in this example, 46 tiles become energized.

The light isn't energizing enough tiles to produce lava; to debug the contraption, you need to start by analyzing the current situation. **With the beam starting in the top-left heading right, how many tiles end up being energized?**

In [1]:
from utilities import get_lines
import pandas as pd
from collections import namedtuple

In [2]:
layout = pd.DataFrame([[ch for ch in line] for line in get_lines('input',16)])

In [3]:
r,c = layout.shape

In [4]:
r,c = layout.shape
def reset_energy():
    return pd.DataFrame(0,index=pd.RangeIndex(r),columns=pd.RangeIndex(c))
energy = reset_energy()

def reset_splitters():
    return pd.DataFrame(0,index=pd.RangeIndex(r),columns=pd.RangeIndex(c))

splitters = reset_splitters()

In [5]:
Beam = namedtuple('Beam', 'row col dir')

In [6]:
def move_down(beam):
    row = beam.row+1
    return Beam(row, beam.col, 'S')

def move_up(beam):
    row = beam.row-1
    return Beam(row, beam.col, 'N')

def move_left(beam):
    col = beam.col-1
    return Beam(beam.row, col, 'W')

def move_right(beam):
    col = beam.col+1
    return Beam(beam.row, col, 'E')

In [7]:
def update_energy(energy, beam):
    if (0<=beam.row<r)&(0<=beam.col<c):
        val = energy.loc[beam.row,beam.col]
        energy.at[beam.row, beam.col] = 8 #  += 1 Don't need to keep adding values
        return True
    return False

In [8]:
def get_action(beam):
    return layout.loc[beam.row,beam.col]

In [9]:
keep_going = {'E':move_right,
              'W':move_left,
              'N':move_up,
              'S':move_down
             }

back_mirror = {'E':move_down,
               'W':move_up,
               'N':move_left,
               'S':move_right
              }

forward_mirror = {'E':move_up,
                  'W':move_down,
                  'N':move_right,
                  'S':move_left
                 }

In [10]:
def get_splitter_state(beam):
    r,c = beam.row, beam.col
    return splitters.loc[r,c]

In [11]:
def get_180_dir(dir):
    compass = 'NESW'
    i = compass.index(dir)
    return compass[(i+2)%4]

In [12]:
def get_90_dir(dir):
    compass = 'NESW'
    i = compass.index(dir)
    return compass[(i+1)%4]

In [13]:
def mark_used_splitter(beam):
    splitters.loc[beam.row, beam.col]=1 #  mark splitter as used
    return None

In [14]:
def split_beam(beam):
    dirs = 'NESW'
    opp_dir = get_180_dir(beam.dir)
    new_beam = Beam(beam.row, beam.col, opp_dir)
    # print(beam, 'and new beam', new_beam)
    return new_beam

In [15]:
def split_vert(beam): #  | splitter
    if (beam.dir=='N')|(beam.dir=='S'):
        return keep_going[beam.dir](beam)
    else: # splitter actions are to mark splitters layout and then either create a new beam or stop
        if get_splitter_state(beam)==0:
            beam = Beam(beam.row, beam.col, get_90_dir(beam.dir)) # make a right turn
            beams.append(split_beam(beam)) #  create a new beam 
            mark_used_splitter(beam) #  mark the splitter as used
            return keep_going[beam.dir](beam)
        else:
            return Beam(-1,-1,beam.dir) #  beam is off the layout...will stop
            

In [16]:
def split_horiz(beam): #  - splitter
    if (beam.dir=='E')|(beam.dir=='W'):
        return keep_going[beam.dir](beam)
    else: # splitter actions are to mark splitters layout and then either create a new beam or stop
        if splitters.loc[beam.row, beam.col]==0:
            beam = Beam(beam.row, beam.col, get_90_dir(beam.dir))
            beams.append(split_beam(beam)) #  create a new beam
            mark_used_splitter(beam) #  mark the splitter as used
            return keep_going[beam.dir](beam)
        else:
            return Beam(-1,-1,beam.dir) #  force stop, splitter has already been used

In [17]:
def take_step(beam):
    action = get_action(beam)
    if action=='/': #  forward slash mirror, nicknamed forward mirror
        beam = forward_mirror[beam.dir](beam)
    elif action=='\\': #  back slash mirror, nicknamed back mirror
        beam = back_mirror[beam.dir](beam)
    elif action=='.':
        beam = keep_going[beam.dir](beam)
    elif action=='|':
        beam = split_vert(beam)
    elif action=='-':
        beam = split_horiz(beam)
    return beam

In [18]:
def activate_beam(beam):
    on_layout = update_energy(energy, beam)
    while on_layout:
        beam = take_step(beam)
        on_layout = update_energy(energy, beam)
    

In [19]:
energy = reset_energy()
splitters = reset_splitters()
beams = [Beam(0,0,'E')]

In [20]:
max_beams = 1
total_beams = 1
while len(beams)>0:
    #  This is just informative
    if len(beams)>max_beams:
        max_beams=len(beams)
       
    beam = beams.pop()

    #  Also just informative
    total_beams+=1
    
    activate_beam(beam)

The `sum` function on the dataframe failed me. I don't know what it thought it was doing. In any case, the solution is not `sum(energy>0)`! 

Doing it the very old-school way:

In [21]:
total = 0
for i in range(r):
    for j in range(c):
        if energy.loc[i,j]!=0:
            total+=1
print(f'The number you are lookin for is {total}.')
print(f'The maximum beams waiting at one time was {max_beams}.')
print(f'The total number of beams was {total_beams}.')

The number you are lookin for is 7067.
The maximum beams waiting at one time was 55.
The total number of beams was 214.
